In [1]:
%%javascript
require(["codemirror/keymap/sublime", "notebook/js/cell", "base/js/namespace"],
    function(sublime_keymap, cell, IPython){
        cell.Cell.options_default.cm_config.keyMap = 'sublime';
        var cells = IPython.notebook.get_cells();
        for(var cl=0; cl< cells.length ; cl++){
            cells[cl].code_mirror.setOption('keyMap', 'sublime');
        }
    }
);

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
pip install git+https://github.com/demetriomagatti/bioinformatics_basics.git@main

  Cloning https://github.com/demetriomagatti/bioinformatics_basics.git (to revision main) to /tmp/pip-req-build-s23fn0cz
  Running command git clone -q https://github.com/demetriomagatti/bioinformatics_basics.git /tmp/pip-req-build-s23fn0cz
Note: you may need to restart the kernel to use updated packages.


### Example

In [4]:
from bioinformatics_basics import alignment
solver = alignment.ManhattanSolver(nrows=5,ncols=5,MaxScore=10)

Initialized module


In [5]:
# matrix containing scores for "to nord" movements
solver.MoveNord

array([[6., 8., 8., 2., 9.],
       [1., 2., 1., 8., 2.],
       [0., 8., 4., 8., 8.],
       [9., 4., 5., 3., 0.]])

In [6]:
# matrix containing scores for "to east" movements
solver.MoveEast

array([[4., 5., 2., 5.],
       [6., 4., 9., 7.],
       [4., 5., 8., 1.],
       [9., 6., 3., 5.],
       [2., 5., 2., 7.]])

In [7]:
# matrix containing maximum score to a vertex - empty before running
solver.Score

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [8]:
# moves to maximize score to a vertex - empty before running
solver.Moves

{}

In [9]:
solver.run()

In [10]:
# matrix containing maximum score to a vertex
solver.Score

array([[ 0.,  4.,  9., 11., 16.],
       [ 6., 12., 17., 26., 33.],
       [ 7., 14., 19., 34., 35.],
       [ 7., 22., 28., 42., 47.],
       [16., 26., 33., 45., 52.]])

In [11]:
# moves to maximize score to the final vertex
solver.Moves[list(solver.Moves.keys())[-1]]

['East', 'East', 'Nord', 'East', 'Nord', 'Nord', 'Nord', 'East']

In [13]:
# visual check
solver.visualize()

NameError: name 'plt' is not defined